In [ ]:
%matplotlib notebook

import functools

import matplotlib.pyplot as plt
import matplotlib.animation
import numpy as np
import xarray as xr

import lifetimes


def animate(data: xr.DataArray, time_coordinate: str = "time") -> None:
    times = np.datetime_as_string(data[time_coordinate])
    fig = plt.figure()
    im_plot = plt.imshow(data.isel({time_coordinate: 0}).values, vmin=np.min(data), vmax=np.max(data))
    ax = plt.gca()
    title = ax.text(0.5, 1.100, f"time step {times[0]}", transform=ax.transAxes, ha="center")

    def animation(i):
        im_plot.set_data(data.isel({time_coordinate: i}).values)
        title.set_text(f"time step {times[i]}")
        return [im_plot]

    anim = matplotlib.animation.FuncAnimation(fig, animation, frames=len(data[time_coordinate]))
    plt.show()
    return anim

In [ ]:
# Create fake dataset of two temporarily variable elliptical data regions on a grid
dataset = lifetimes.testing.create_dummy_ecmwf_ifs_hres_dataset(
    grid_size=(10, 10)
)
ds = dataset.as_xarray()

# Or load from local file
#path = '/home/fabian/Documents/MAELSTROM/data/ml_20190101_00.nc'
#ds = xr.open_dataset(path).isel(level=0)

data = ds["ellipse"]

# determing modes (perform spatio-temporal PCA)
modes = [lifetimes.modes.Modes(feature=data)]
pca_partial_method = functools.partial(
    lifetimes.modes.methods.spatio_temporal_principal_component_analysis,
    time_coordinate="time",
    x_coordinate=None,
    y_coordinate=None,
    variance_ratio=None,
)
[pca] = lifetimes.modes.determine_modes(modes=modes, method=pca_partial_method)
data.shape

In [ ]:
anim = animate(data)

In [ ]:
pca_da = xr.DataArray(data=pca.components_)
pca_da.plot()

In [ ]:
original_shape = pca.components_.reshape(data.shape)
original_shape.shape

In [ ]:
ds["pcs"] = (["time", "latitude", "longitude"], original_shape)
pca.components_.shape

In [ ]:
anim = animate(ds["pcs"])
plt.show()

In [ ]:
data_weighted = lifetimes.utils.weight_by_latitudes(data, latitudes="latitude")
data_standardized = lifetimes.utils.standardize(data_weighted)

In [ ]:
anim = animate(data_standardized)
plt.plot()

In [ ]:
data_reshaped = lifetimes.utils.reshape_spatio_temporal_numpy_array(data_standardized.values)

In [ ]:
d, theta = lifetimes.modes.methods.indicators(
    data_reshaped, 
    metric="euclidean", 
    q=0.97, 
    pareto_fit="scipy", 
    theta_fit="sueveges", 
)

indicators = xr.Dataset(
    data_vars={
        "d": (["time"], d),
        "theta": (["time"], theta),
    },
    coords={"time": data.coords["time"]},
)

In [ ]:
indicators["d"].plot()

In [ ]:
indicators["theta"].plot()